# Travelling Salesman Problem using Genetic algorithm
## **By Karthika Nair and Changlong Ji M1-CSN**

In [22]:
import random
import math


# Generate the distance matrix
def generate_map(num_cities):
    cities = [i for i in range(1,num_cities+1)]
    distances = [[0] * num_cities for _ in range(num_cities)]
    for i in range(num_cities):
        for j in range(i+1, num_cities):
            # Generate random distances, if two cities are not connected, the distance is set to positive infinity
            distance = random.randint(1, num_cities)
            if random.random() < 0.1:  # 10% probability to set distance to positive infinity
                distance = math.inf
            distances[i][j] = distance
            distances[j][i] = distance  # Set the distance in the symmetric position
    return distances
            
# Output the distance matrix
distances = generate_map(num_cities = 20)
def print_map(distances):
    for row in distances:
        print(row)


1. Encoding and creating a population

In [8]:
from random import sample

import random


def create_population(population_size = 5,num_cities = 10):
    population = []
    for i in range(population_size):
        new_path = sample(cities, len(cities))
    # 如果路径已经存在于种群中，则重新生成路径
        while new_path in population:
            new_path = sample(cities, len(cities))
        population.append(new_path)
    return population


In [ ]:
# from random import shuffle
# cities = [1,2,3,4,5]
# shuffle(cities)
# print(cities)

Mutation

2. Evaluation:
Calculate the distance of each population

In [17]:
def get_path_distance(path, distances):
    distance = 0
    for i in range(num_cities):
        current_city = path[i]
        next_city = path[(i+1) % num_cities]
        if next_city == path[0] and i == num_cities - 1:
            distance += distances[current_city-1][next_city-1]
        else:
            distance += distances[current_city-1][next_city-1]
    return distance
def get_distance(population,distances):
    dis = []
    for i in population:
        dis.append(get_path_distance(i, distances))
    return dis 

3. Crossover

In [10]:
def find_parents(population, distances):
    population2 = [i for i in population]
    parent1 = min(population2, key=lambda x: get_path_distance(x, distances))
    population2.remove(parent1)
    parent2 = min(population2, key=lambda x: get_path_distance(x, distances))
    return(parent1,parent2)

In [11]:
import random

def crossover2(p1, p2):
    p11 = [i for i in p1]
    start = random.randint(0, len(p1)-1)
    child = p2
    used_indexes = []
    p11[start] = 0
    element = p2[start]
    used_indexes.append(start)
    while(element in p11):
        used_indexes.append(p11.index(element))
        p11[p11.index(element)] = 0
        element = p2[p1.index(element)]
    for i in used_indexes:
        child[i] = p1[i]
    return child



def crossover(p1,p2):
    a = p1
    b = p2
    c=[0]*num_cities
    key_list=[]
    key=random.choice(a)
#     print("The initial point in Parent 1 is: ",key)
    j=0
    while j < len(a):
        i=0
        while i < len(a):
            #checking if the key element is found in parent1
            if a[i] == key:
                    c[i]=a[i]
                    key_list.append(key) #append the key to visited key list
                    key=b[i]
            #break if the key is in key_list         
            if key in key_list:
                break
            i=i+1  
        j=j+1
    #print(key_list) 
    for i in c:
        if i == 0:
            c[c.index(i)]=b[c.index(i)]
    return c


4. Mutation

In [12]:
from math import ceil
def mutate(child,least = 0.2, most = 0.5):   
    mutate_index = random.randint(0, num_cities-1)
    mutate_num = random.randint( ceil(num_cities*least), ceil(num_cities*most))
    random_direction = random.choice([-1, 1])
    mutate_index1 = min(mutate_index,mutate_index+(random_direction)*mutate_num)
    mutate_index2 = max(mutate_index,mutate_index+(random_direction)*mutate_num)
    if mutate_index1 <0:
        mutate_index1 = 0
    if mutate_index2 > num_cities-1:
        mutate_index2 = num_cities
    child[mutate_index1:mutate_index2+1] = reversed(child[mutate_index1:mutate_index2+1])
    return child


validate if child in population

In [13]:
def validate_child(child,population):
    for i in range(0,10):
        if child in population:
            child = mutate(child)
        else: break
    while(child in population):
        child = mutate(child,0.5,1)
    population.append(child)
    return population

Loop and get a local minumal answer

In [25]:
num_cities = 20
population_size = 10
ditsances = generate_map(num_cities)
print_map(distances)
population = create_population(population_size)
for i in range(0,200):
    dis = get_distance(population,distances)
    p1,p2 = find_parents(population, distances)
    child = crossover(p1, p2)
    population = validate_child(child,population)
    dis = get_distance(population,distances)
    print(min(dis))

[0, 4, 12, 12, 17, 16, 6, 9, 1, 11, 11, 3, 5, 4, 3, 9, 13, 6, 8, 10]
[4, 0, 4, 2, 13, 1, 4, 13, 8, 14, 6, 10, 13, 19, 4, 1, 16, 17, 1, 18]
[12, 4, 0, 11, 12, 19, 11, 1, inf, 20, 9, 16, 3, 11, 7, 14, 11, 7, 4, 11]
[12, 2, 11, 0, 2, 14, 11, 2, inf, 8, 15, inf, 20, 11, 8, 15, 1, 3, 16, 8]
[17, 13, 12, 2, 0, 11, 17, 6, 17, 7, 15, 5, inf, 12, 17, 18, 10, 1, 18, inf]
[16, 1, 19, 14, 11, 0, 11, 1, inf, 1, 20, 2, 2, inf, 7, 5, 7, inf, 6, 9]
[6, 4, 11, 11, 17, 11, 0, 18, 3, inf, 3, 14, 15, 14, 7, 10, 6, 4, 19, inf]
[9, 13, 1, 2, 6, 1, 18, 0, 15, 8, 8, inf, 6, 18, 9, 19, 10, inf, inf, inf]
[1, 8, inf, inf, 17, inf, 3, 15, 0, 9, inf, 15, 20, 14, 19, 16, 16, 5, 3, 15]
[11, 14, 20, 8, 7, 1, inf, 8, 9, 0, 12, 18, 18, 14, 20, inf, 15, 7, 4, 15]
[11, 6, 9, 15, 15, 20, 3, 8, inf, 12, 0, 9, 2, 10, 13, 12, 7, 16, 8, 11]
[3, 10, 16, inf, 5, 2, 14, inf, 15, 18, 9, 0, 3, 16, inf, 9, 14, inf, 7, 19]
[5, 13, 3, 20, inf, 2, 15, 6, 20, 18, 2, 3, 0, 11, 18, 20, inf, 7, inf, 3]
[4, 19, 11, 11, 12, inf, 14, 18, 14

For Karthika: you can go on, and if u have any problems just connect me! ^^